In [3]:
import numpy as np
import logging
import pickle

logging.basicConfig(filename=f'queue_state.log', filemode='w', format='%(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
def conditions_generation(l: int, mu: int, size: int) -> tuple[np.ndarray, np.ndarray]:
    return [[np.random.exponential(scale=1/l, size=1)[0], np.random.exponential(1/mu, size=1)[0]] for _ in range(size)]

In [5]:
l = 3
mu = 1
size =100
'T1 - время между приходами i-1 и i требований'
'T2 - время обсуживания i-ого требования'

conditions = conditions_generation(l=l, mu=mu, size=size)
conditions[0][0] = 0
conditions

[[0, 2.4405946573748976],
 [0.005789998758006235, 1.1600305256129613],
 [0.004921361978384275, 0.31071279230397736],
 [0.22037554973661805, 3.301303390041197],
 [0.21702576073674945, 0.3606157926888402],
 [0.12873501921749395, 0.11517839576978196],
 [0.008296205847502123, 0.026374565074543232],
 [0.021914594550911277, 1.0451696796746937],
 [0.03134461239545951, 0.37469855158919835],
 [0.004414681943459441, 0.040414436261467616],
 [0.08379958846579821, 0.5450914122416609],
 [0.10392284214704253, 0.07318154193749632],
 [0.6587013225732464, 0.11142840968755027],
 [0.07379390134454238, 0.5125650280499725],
 [0.07093203338275204, 0.6385320091748748],
 [0.08432988705618459, 0.7879140385596568],
 [0.07301537638752012, 0.41805393943653074],
 [0.022054150787281068, 0.2758575552081723],
 [0.01876160350050353, 1.8026610076896976],
 [0.023251366467851066, 1.3233382976362495],
 [0.005067370940404597, 0.6928606746922863],
 [0.039966726829196754, 0.056723301143390394],
 [0.14521913293280111, 0.044100

In [6]:
queue_capacity = 5

def model(max_queue_capacity: int, conditions: tuple):

    current_time = 0
    remaining_server_time = 0
    current_queue_occupancy = []
    
    time_log = []
    queue_log = []
    server_log = []
    
    
    while len(conditions) != 0:
        
        t1, t2 = conditions.pop(0)
        logger.info(f"Current itteration t1: {t1} - t2: {t2}")
        logger.info(f"Before itter. Server state: {remaining_server_time}, Queue state: {current_queue_occupancy}")
        
        if t1 >= sum(current_queue_occupancy) + remaining_server_time:
            t1 -= remaining_server_time
            
            current_time += remaining_server_time           
            
            while len(current_queue_occupancy) != 0 and t1 > 0:
                next_t2 = current_queue_occupancy.pop(0)
                t1 -= next_t2
                
                current_time += next_t2
                
                time_log.append(current_time)
                queue_log.append(current_queue_occupancy)
                server_log.append(remaining_server_time)  

            if t1 > 0:
                current_time += t1
            
            remaining_server_time = t2

        else:
            
            if remaining_server_time > t1:
                remaining_server_time -= t1
                current_time += t1
                
                if len(current_queue_occupancy) == max_queue_capacity:
                    logger.info(f"Queue is full. Drop this operations.\n")
                else:
                    current_queue_occupancy.append(t2)

                time_log.append(current_time)
                queue_log.append(current_queue_occupancy)
                server_log.append(remaining_server_time)
                

            else:
                t1 -= remaining_server_time
                current_time += remaining_server_time
                
                while len(current_queue_occupancy) != 0 and t1 != 0:
                    
                    next_t2 = current_queue_occupancy.pop(0) 

                    time_log.append(current_time)
                    queue_log.append(current_queue_occupancy)
                    server_log.append(remaining_server_time)                    
                    
                    if t1 - next_t2 > 0:
                        
                        t1 -= next_t2
                        current_time += next_t2

                    else:
                        
                        next_t2 -= t1
                        current_time += next_t2                    
                        t1 = 0
                                                
                current_queue_occupancy.append(t2)
                
                time_log.append(current_time)
                queue_log.append(current_queue_occupancy)
                server_log.append(remaining_server_time) 
                
        with open('time.pkl', 'wb') as tf:
            pickle.dump(time_log, tf)
        with open('queue.pkl', 'wb') as qf:
            pickle.dump(queue_log, qf)    
        with open('server.pkl', 'wb') as sf:
            pickle.dump(server_log, sf)
            
        logger.info(f"After itter. Server state: {remaining_server_time}, Queue state: {current_queue_occupancy}\n")
    

In [7]:
model(max_queue_capacity=queue_capacity, conditions=conditions)